In [1]:
import os
from mp_api.client import MPRester
os.environ["MAPI_API_KEY"] = "MY_API_KEY"
mpr = MPRester(os.getenv("MAPI_API_KEY")) 

/Users/samcox/anaconda3/envs/mapi/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from exact_matching import ExactMatchingTool
exact_matching = ExactMatchingTool()

### Case 1: We know the formula, density, and the symmetry info (lattice & space group)

In [4]:
formula="LiCoO2"
space_group = 12
lattice="Monoclinic"
density=4.7
prop="band_gap"

input_dict_full = {"formula": formula, "space_group_num": space_group, "lattice": lattice, "density": density, "desired_prop": prop}
results = exact_matching.query_material_property(**input_dict_full, one_only=False)

In [5]:
results

[MPDataDoc<SummaryDoc>(
 formula_pretty='LiCoO2',
 density=4.6776427564172245,
 symmetry=SymmetryData(crystal_system=<CrystalSystem.mono: 'Monoclinic'>, symbol='C2/m', number=12, point_group='2/m', symprec=0.1, version='2.0.2'),
 band_gap=2.0128000000000004,
 fields_not_requested=['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_anonymous', 'chemsys', 'volume', 'density_atomic', 'property_name', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetizat

In [6]:
#just the property
print (f"Property {prop}: {exact_matching._get_prop(results, prop)}")

Property band_gap: [2.0128000000000004]


### Case 2: We know less information

In [7]:
formula="LiCoO2"
#space_group = 12
lattice="Monoclinic"
#density=4.7
prop="band_gap"

input_dict_missing = {"formula": formula, "desired_prop": prop}
results = exact_matching.query_material_property(**input_dict_missing, one_only=False)

In [8]:
len(results)

9

In [9]:
#hm...there are 9 resutls. Lets get the average overal all of them

average_prop = exact_matching.average_vanilla(results, prop)
print (f"Property {prop}: {average_prop}")

Property band_gap: 0.8278000000000001


### Case 3: There are no results for what we want to know about

In [10]:
formula = "LiFePO4"
space_group = 250
lattice = "Cubic"
density = 4.87
prop = "band_gap"

input_dict_no_results = {"formula": formula, "space_group_num": space_group, "lattice": lattice, "density": density, "desired_prop": prop}

results = exact_matching.query_material_property(**input_dict_no_results, one_only=False)

In [11]:
results

[]

In [12]:
#Hm...its blank. Lets try to get the average of all the nearest neighbors
nearest_neighbor_avg = exact_matching.get_weighted_average_of_neighbors(input_dict_no_results)

In [13]:
print (f"Property {prop}: {nearest_neighbor_avg}")

Property band_gap: 3.1868915094339627


### Putting it all together

In [14]:
#case 1:
formula="LiCoO2"
space_group = 12
lattice="Monoclinic"
density=4.7
prop="band_gap"

input_dict_full = {"formula": formula, "space_group_num": space_group, "lattice": lattice, "density": density, "desired_prop": prop}

results = exact_matching.get_prop(**input_dict_full)
print (f"Property {prop}: {results}")

Property band_gap: 2.0128000000000004


In [15]:
#case 2:
formula="LiCoO2"
lattice="Monoclinic"
prop="band_gap"

input_dict_missing = {"formula": formula, "desired_prop": prop}

results = exact_matching.get_prop(**input_dict_missing)
print (f"Property {prop}: {results}")

Property band_gap: 0.8278000000000001


In [16]:
#case 3:
formula = "LiFePO4"
space_group = 250
lattice = "Cubic"
density = 4.87
prop = "band_gap"

input_dict_no_results = {"formula": formula, "space_group_num": space_group, "lattice": lattice, "density": density, "desired_prop": prop}

results = exact_matching.get_prop(**input_dict_no_results)
print (f"Property {prop}: {results}")

Property band_gap: 3.1868915094339627
